In [1]:
from __future__ import annotations # To prevent Subscript for class "list" will generate runtime exception; enclose type annotation in quotes
import seaborn as sns
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import scipy
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from Kmeans import *
from decimal import Decimal
from SVD import * 
%matplotlib inline
# pd.options.display.max_rows = 4000
# pd.options.display.max_columns = 4000

In [2]:
df = pd.read_csv('preprocessed.csv',index_col=0)
Genres=list(pd.read_csv('genre_order.csv',index_col=0)['0'])
df

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Action,1920s,Animation,Comedy,Thriller,Mystery,1950s,Children's,Film-Noir,Drama
UserID,,,,,,,,,,,,,,,,,,,,,
1,0,0.017857,0,0,0,1,0,0,0,0,...,0.182449,0.000000,0.645186,0.504759,0.091603,7.379219e-02,0.125432,0.747438,0.004701,0.822336
2,1,1.000000,0,0,0,0,0,0,1,0,...,0.607944,0.014101,0.027614,0.273131,0.330404,2.202718e-02,0.061674,0.057179,0.004405,1.000000
3,1,0.446429,0,0,0,0,1,0,0,0,...,0.814808,0.023769,0.086422,1.000000,0.148147,6.255649e-07,0.205044,0.086422,0.125994,0.271601
4,1,0.803571,0,1,0,0,0,0,0,0,...,0.354289,0.049776,0.194632,1.000000,0.047685,1.704221e-01,0.205070,0.004760,0.104962,0.102874
5,1,0.446429,0,0,0,0,1,0,0,0,...,0.125002,0.005014,0.026040,0.346350,0.182293,3.906287e-02,0.013019,0.039062,0.018228,0.562499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0,0.446429,0,0,1,0,0,0,0,0,...,0.294317,0.006772,0.091950,0.552734,0.301040,7.549845e-02,0.113240,0.123925,0.046457,0.898209
6037,0,0.803571,0,0,0,0,0,0,1,0,...,0.257441,0.000000,0.003553,0.535784,0.650177,1.178472e-01,0.246573,0.049999,0.071433,1.000000
6038,0,1.000000,1,0,0,0,0,0,0,0,...,0.062499,0.062497,0.187502,1.000000,0.794578,2.552067e-01,0.156252,0.000000,0.134377,0.750100


In [3]:
num_cluster=26
to_remove=10
threshold=0

In [4]:
from SVD import * 
u,s,sigma,V_trans = ReducedSVD(df.to_numpy(),threshold,num_cluster)

In [5]:
df = pd.DataFrame(u@sigma@V_trans,columns=df.columns)

In [6]:
def cluster_to_matrix(X_train,y_train,num_cluster):
    k=Kmeans(K=num_cluster,iter=150,p=2,random_state=42)
    y_pred,_=k.predict(np.array(X_train),choice=0)
    k.centroids=np.array(k.centroids)
    empty_cluster = []
    for idx,i in enumerate(k.clusters):
        if len(i)==0:
            empty_cluster.append(idx)
    X_train2 = pd.concat([X_train,y_train,pd.DataFrame(y_pred,columns=["label"],dtype='int')],axis="columns")
    df_cluster_rep = X_train2.groupby(['label'], as_index=False).mean()
    df_cluster_rep.drop(labels="label",axis="columns",inplace=True)
    matrix = df_cluster_rep.to_numpy()
    final_matrix = []
    j = 0 
    for i in range(num_cluster):
        if i in empty_cluster:
            final_matrix.append(np.zeros(len(df.columns)))
        else:
            final_matrix.append(matrix[j])
            j+=1

    return np.array(final_matrix),k 

In [7]:
x_col_size=len(df.columns)-len(Genres)+6

In [8]:
x_col_size

38

In [9]:
y = df[df.columns[x_col_size:]]
x = df.drop(labels=df.columns[x_col_size:],axis="columns")

In [10]:
x

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1990s,War,1980s
0,0.336049,0.166830,-0.145981,-0.074633,0.009918,1.022513,-0.046761,0.145341,0.067151,0.005155,...,-0.035969,-0.040216,-0.003829,-0.060699,0.045502,0.280682,0.392179,1.218746,0.145913,0.592313
1,1.595441,1.448911,-0.228231,-0.086227,0.072817,0.082926,-0.044575,0.330858,1.164862,0.066865,...,-0.018632,-0.057777,-0.046371,-0.049488,-0.033890,0.462672,0.027326,1.347000,0.174327,0.900957
2,1.585375,1.006599,-0.242453,-0.074164,0.037016,0.043498,0.947243,0.317604,0.150481,0.043186,...,-0.017743,-0.013726,-0.028743,-0.026501,-0.024413,0.421445,0.103736,1.100516,0.168041,1.297585
3,1.280949,0.980482,-0.133886,0.929321,-0.002001,0.007921,-0.049827,0.120650,0.045723,0.000703,...,-0.032198,-0.024137,-0.035501,-0.053992,0.058961,0.257724,0.100473,0.385589,0.032882,0.526346
4,1.365547,0.756134,-0.161598,-0.071647,0.020250,0.027114,0.950114,0.177245,0.079127,0.017037,...,-0.021550,-0.015412,-0.014955,0.958113,-0.008637,0.241186,0.033647,1.244274,0.020480,0.385197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.425182,0.881848,-0.187840,-0.063835,1.008697,0.014994,-0.054054,0.229009,0.094619,0.025067,...,-0.010782,-0.032540,-0.011149,-0.025071,-0.049546,0.376873,0.063152,1.266180,0.129143,0.839083
6036,0.467456,1.184388,-0.199200,-0.084790,0.042647,0.048639,-0.053817,0.247899,1.111564,0.034294,...,-0.021723,-0.060458,-0.044993,-0.039481,0.020674,0.598126,0.077972,0.853891,0.143823,0.857997
6037,0.452407,1.321721,0.837579,-0.050789,0.067992,0.078426,-0.011790,0.260581,0.135133,0.061530,...,0.003397,-0.058658,-0.045196,-0.022379,0.097737,0.272780,0.245861,0.844004,0.091775,0.876919
6038,0.282863,1.050870,0.874176,-0.049283,0.028022,0.033546,-0.026019,0.159328,0.074181,0.026853,...,-0.013054,-0.048772,-0.039967,-0.020210,0.109999,0.265889,0.303366,0.543464,0.138686,0.553537


In [11]:
y

,Western,Horror,1960s,1910s,1930s,Crime,Romance,Fantasy,1940s,Adventure,...,Action,1920s,Animation,Comedy,Thriller,Mystery,1950s,Children's,Film-Noir,Drama
0,0.087530,0.277235,-0.039723,-0.014952,-0.029916,0.280738,0.443464,0.377683,0.121333,0.521059,...,0.245665,-0.022945,0.504631,0.623230,0.247366,0.127101,0.110667,0.835700,0.124922,1.131766
1,0.090835,0.077991,-0.101835,-0.034351,-0.138646,0.401005,0.574452,0.223613,0.132880,0.668955,...,0.748555,-0.062986,-0.080671,0.384339,0.629510,0.144908,0.097091,0.106760,0.172980,1.486164
2,0.125348,0.190903,-0.044967,-0.042330,-0.130188,0.352123,0.373590,0.330760,0.141075,0.930211,...,1.192800,-0.076271,0.015580,1.134754,0.495482,0.122322,0.078716,0.285105,0.164683,0.773122
3,0.106567,-0.027700,0.081417,0.002949,0.036392,0.204590,0.222836,0.169398,0.228046,0.330638,...,0.300385,0.030317,0.006771,0.961757,0.204604,0.159129,0.158070,0.105014,0.189227,0.249544
4,0.051806,0.108017,-0.099504,-0.019258,-0.093769,0.256769,0.331172,0.148304,0.059713,0.319806,...,0.224088,-0.043629,-0.033760,0.440031,0.403911,0.062761,0.027104,0.080386,0.094956,0.881001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6035,0.036276,0.318051,-0.054508,-0.035205,-0.121755,0.297151,0.374012,0.239904,0.051030,0.520540,...,0.502602,-0.074660,0.017820,0.668664,0.492675,0.082765,0.042450,0.222210,0.091704,1.284954
6036,0.094675,0.198238,-0.011934,-0.028220,-0.052371,0.497162,0.461634,0.199421,0.186553,0.424310,...,0.425192,-0.037885,-0.071866,0.622413,0.748655,0.236139,0.159066,0.035623,0.226715,1.394072
6037,0.155847,0.153834,0.029372,-0.009164,0.044963,0.659976,0.754790,0.226246,0.361505,0.268113,...,0.304345,0.007909,0.055021,1.101155,0.924828,0.399862,0.271404,0.202289,0.374482,1.176510
6038,0.154863,0.043865,0.215385,0.001340,0.084782,0.305265,0.674585,0.242897,0.324157,0.380883,...,0.198850,0.025187,0.160467,1.015050,0.278574,0.243956,0.310728,0.326105,0.255935,0.679445


In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.05, shuffle=True, random_state=42)

In [13]:
matrix,kmeans_model = cluster_to_matrix(x_train,y_train,num_cluster)

In [14]:
matrix.shape

(26, 60)

In [15]:
def predict(datapoint,kmeans,matrix_clean):
    ''' 
        @input: Takes in a datapoint, kmeans object, and information matrix 
        @output: Returns: rating sorted with index for column name mapping
    '''
    y_pred = kmeans.predictPoint(datapoint)
    inferred_full = matrix_clean[y_pred]
    inferred = inferred_full[x_col_size:]
    inferred_full = inferred_full[len(df.columns)-len(Genres):]
    sorted_ratings = []
    sorted_ratings_full = []
    for idx,element in enumerate(inferred):
        sorted_ratings.append([element,idx+x_col_size])
    for idx,element in enumerate(inferred_full):
        sorted_ratings_full.append([element,idx+len(df.columns)-len(Genres)])
    sorted_ratings.sort()
    sorted_ratings.reverse()
    sorted_ratings_full.sort()
    sorted_ratings_full.reverse()
    return sorted_ratings, sorted_ratings_full

In [16]:
x_test.to_numpy()[0][:x_col_size]
sorted_ratings, sorted_ratings_full = predict(x_test.to_numpy()[0][:x_col_size],kmeans_model,matrix)
s="Top 5 PREDICTED recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_full[i][1]]+"\n"
print(s)

Top 5 PREDICTED recommended genres are:
1990s
Drama
1980s
Comedy
Action



In [17]:
sorted_ratings_actual=[]
inferred = y_test.to_numpy()[0]
for idx,element in enumerate(inferred):
    sorted_ratings_actual.append([element,idx+x_col_size])
sorted_ratings_actual.sort()
sorted_ratings_actual.reverse()
s="Top 5 ACTUAL recommended genres are:\n"
for i in range(5):
    s+=df.columns[sorted_ratings_actual[i][1]]+"\n"
print(s)

Top 5 ACTUAL recommended genres are:
Drama
Comedy
Romance
Adventure
Thriller



In [18]:
rating_dict={}
for i in range(len(sorted_ratings_full)):
    rating_dict[df.columns[sorted_ratings_full[i][1]]]=sorted_ratings_full[i][0]
rating_dict

{'1990s': 1.0643013544185944,
 'Drama': 1.029359288285216,
 '1980s': 0.8232468419582389,
 'Comedy': 0.6433171355614216,
 'Action': 0.5640417270601236,
 'Adventure': 0.5609571783501245,
 'Thriller': 0.5543229104362375,
 'Sci-Fi': 0.4865571792409807,
 'Romance': 0.4660780189251096,
 '1970s': 0.3939639371188012,
 'Crime': 0.3810160374394764,
 '2000s': 0.3205962522593354,
 'Fantasy': 0.2798340052208155,
 'Horror': 0.23689226070561822,
 "Children's": 0.20316539659304628,
 'Film-Noir': 0.19151806370877422,
 '1940s': 0.1774811489914049,
 'Mystery': 0.1772458784257307,
 '1950s': 0.13563224750171526,
 'Musical': 0.12180097635076434,
 'War': 0.11910295530906757,
 'Western': 0.11020326761805466,
 'Animation': 0.0382486988671253,
 'Documentary': 0.028864386675302684,
 '1960s': -0.016013839801089794,
 '1910s': -0.022782188586771907,
 '1920s': -0.0324016583037522,
 '1930s': -0.059505291418322344}

In [19]:
def predict_for_dataset(X_test,kmeans_instance,matrix_clean):
    X_test = X_test.to_numpy()
    y_pred = []
    for i in range(X_test.shape[0]):
        datapoint = X_test[i]
        y_pred.append(predict(datapoint,kmeans_instance,matrix_clean)[0])
    return np.array(y_pred)

In [20]:
x_test

,Gender,Age,Zip-code_1,Zip-code_2,Zip-code_3,Zip-code_4,Zip-code_5,Zip-code_6,Zip-code_7,Zip-code_8,...,Occupation_17,Occupation_18,Occupation_19,Occupation_20,Documentary,Sci-Fi,Musical,1990s,War,1980s
5529,0.481496,0.724274,-0.200913,-0.081826,0.040749,0.049156,-0.051307,1.244113,0.117259,0.032422,...,-0.021136,-0.063838,-0.041260,-0.041802,-0.022275,0.306892,0.083740,1.302837,0.040535,0.825667
710,1.535622,0.979626,-0.226639,-0.082167,0.047175,0.052262,-0.041729,0.300137,0.139651,1.046610,...,0.021861,-0.041157,-0.029685,0.018863,-0.026600,0.459453,0.046197,0.954382,0.136871,1.109118
4923,0.354115,0.871897,-0.151923,-0.070540,0.015501,0.024605,-0.047102,0.164749,0.073985,0.014697,...,-0.021549,-0.043015,-0.032829,-0.044151,0.060026,0.190852,0.092503,0.903949,0.108115,0.654296
2153,1.594347,0.930584,-0.234831,-0.090825,0.071988,0.076077,-0.052377,1.314708,0.154887,0.061421,...,-0.014204,-0.063295,-0.036085,-0.048322,0.050641,0.539918,0.159047,1.321795,0.141239,0.919344
1272,1.440498,1.017867,0.828439,-0.061080,0.058240,0.062889,-0.031763,0.250993,0.126535,0.047581,...,-0.016660,-0.037481,-0.018797,-0.035200,0.016072,0.307975,0.077451,1.024688,0.110797,0.859832
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3341,0.219379,0.674996,-0.116895,-0.065787,0.001743,0.021722,-0.041800,0.110191,0.025721,0.006178,...,0.015269,-0.001951,-0.000211,0.018812,0.084044,0.232512,0.057161,1.166200,-0.005956,0.338270
4648,1.441390,1.542715,0.815103,-0.070346,0.073366,0.055674,-0.037467,0.222833,0.119132,0.027565,...,-0.023923,-0.005830,-0.035586,-0.048370,0.013641,0.272304,0.128153,0.425150,0.241930,0.603460
323,1.460419,1.062869,-0.186224,-0.066761,0.065253,0.070011,0.970391,0.281064,0.137221,0.060591,...,0.991284,-0.037480,-0.040850,-0.021054,0.065547,0.471800,0.243789,1.350949,0.115110,0.611012
2943,0.462321,0.718383,-0.174927,-0.071535,0.051240,1.057374,-0.038579,0.257114,0.124780,0.053478,...,-0.015046,-0.084262,-0.054441,-0.044403,0.034013,0.511189,0.295727,1.240396,0.087054,0.955713


In [21]:
y_pred = predict_for_dataset(x_test,kmeans_model,matrix) 

In [22]:
y_pred=np.array(y_pred)
y_test=np.array(y_test)

In [23]:
y_test.shape

(302, 22)

In [24]:
y_pred_arr=[]
for i in y_pred:
    temp=[0 for _ in range(y_test.shape[1])]
    for j in i:
        temp[int(j[1]-x_col_size)]=j[0]
    y_pred_arr.append(temp)
y_pred_arr=np.array(y_pred_arr)
y_pred_arr.shape


(302, 22)

In [25]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred_arr)

0.027972711656741235

In [26]:
ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")

/tmp/ipykernel_23252/2496518303.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings=pd.read_csv('ml-1m/ratings.dat', delimiter='::', names=['UserID','MovieID','Rating','TimeStamp'])
/tmp/ipykernel_23252/2496518303.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies=pd.read_csv('ml-1m/movies.dat', delimiter='::', names=["Title","Genres"],  encoding="ISO-8859-1")


In [27]:
# movies
ratings.drop(['UserID','TimeStamp'], axis=1, inplace= True)

In [28]:
ratings

,MovieID,Rating
0,1193,5
1,661,3
2,914,3
3,3408,4
4,2355,5
...,...,...
1000204,1091,1
1000205,1094,5
1000206,562,5
1000207,1096,4


In [29]:
movies['Rating']=[[] for _ in range(len(movies.index))]
for i in ratings.index:
    temp=ratings.iloc[i]['Rating']
    movies.at[ratings.iloc[i]['MovieID'],'Rating'].append(temp)

In [ ]:
movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())

/tmp/ipykernel_23190/1822539475.py:1: RuntimeWarning: Mean of empty slice.
  movies['Rating']=movies['Rating'].apply(lambda x: np.array(x).mean())
/home/netradeepakc/anaconda3/envs/rec_sys_mini/lib/python3.12/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [ ]:
movies.isna().sum()

Title       0
Genres      0
Rating    177
dtype: int64

In [ ]:
movies['Genres']=movies['Genres'].apply(lambda x: x.split('|'))
for i in movies.index:
    title=movies.at[i,'Title']
    decade=title[-5:-2]
    decade=str(decade)+'0s'
    movies.at[i,'Genres'].append(decade)

In [ ]:
movies

,Title,Genres,Rating
1,Toy Story (1995),"[Animation, Children's, Comedy, 1990s]",4.146846
2,Jumanji (1995),"[Adventure, Children's, Fantasy, 1990s]",3.201141
3,Grumpier Old Men (1995),"[Comedy, Romance, 1990s]",3.016736
4,Waiting to Exhale (1995),"[Comedy, Drama, 1990s]",2.729412
5,Father of the Bride Part II (1995),"[Comedy, 1990s]",3.006757
...,...,...,...
3948,Meet the Parents (2000),"[Comedy, 2000s]",3.635731
3949,Requiem for a Dream (2000),"[Drama, 2000s]",4.115132
3950,Tigerland (2000),"[Drama, 2000s]",3.666667
3951,Two Family House (2000),"[Drama, 2000s]",3.900000


In [ ]:
movies['Affinity']=[0 for _ in range(len(movies.index))]
for i in movies.index:
    genres=movies.at[i,'Genres']
    affinity=0
    for j in genres:
        affinity+=rating_dict[j]*movies.at[i,'Rating']
    movies.at[i,'Affinity']=affinity

/tmp/ipykernel_23190/3621030468.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.775485378540024' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  movies.at[i,'Affinity']=affinity


In [ ]:
movies.sort_values('Affinity',inplace=True,ascending=False)

In [ ]:
movies

,Title,Genres,Rating,Affinity
3266,Man Bites Dog (C'est arrivé près de chez vous)...,"[Action, Comedy, Crime, Drama, 1990s]",3.655844,12.861748
2000,Lethal Weapon (1987),"[Action, Comedy, Crime, Drama, 1980s]",3.886908,12.457683
2028,Saving Private Ryan (1998),"[Action, Drama, War, 1990s]",4.337354,12.057463
21,Get Shorty (1995),"[Action, Comedy, Drama, 1990s]",3.623894,11.952560
1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Drama, Sci-Fi, War, 1980s]",4.292977,11.936089
...,...,...,...,...
3650,Anguish (Angustia) (1986),"[Horror, 1980s]",NaN,NaN
3750,Boricua's Bond (2000),"[Drama, 2000s]",NaN,NaN
3829,Mad About Mambo (2000),"[Comedy, Romance, 2000s]",NaN,NaN
3856,Autumn Heart (1999),"[Drama, 1990s]",NaN,NaN


In [ ]:
top_movies=list(movies['Title'])[:5]
print("Top 5 movies are:")
for i in top_movies:
    print(i)

Top 5 movies are:
Man Bites Dog (C'est arrivé près de chez vous) (1992)
Lethal Weapon (1987)
Saving Private Ryan (1998)
Get Shorty (1995)
Star Wars: Episode V - The Empire Strikes Back (1980)
